In [70]:
import pandas as pd 
import numpy as np 
import csv
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests
import math

In [104]:
# Reading the healthy and cancerous genes into dataframes
healthy = pd.read_csv('healthy.csv', sep = '\t', index_col=None)
cancerous = pd.read_csv('cancerous.csv', sep = '\t', index_col =None)
healthy     # from here we knew that the dimensions are ( 19648 rows × 51 columns ) and the structure
#cancerous      # to make sure that gens are the same in names and id     
#healthy=healthydf[(healthydf != 0).all(1)]
#cancerous=cancerousdf[(cancerousdf != 0).all(1)]

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,HAVCR2,84868,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19644,RP1-66C13.4,0,0.00,0.00,1.79,3.32,0.00,0.00,1.79,0.00,...,0.00,4.86,0.00,0.00,2.81,2.84,0.00,0.00,1.08,0.00
19645,C3orf79,152118,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.07,0.00,0.00
19646,CTD-2116N17.1,0,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


In [105]:
num=0
for row in range (len(healthy)):
    x=healthy.iloc[row,2:]
    for index in x:
        if index ==0:
            num=num+1
    if num>25:
        healthy=healthy.drop(row)
        num=0        

IndexError: single positional indexer is out-of-bounds

In [106]:
healthy

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,HAVCR2,84868,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19644,RP1-66C13.4,0,0.00,0.00,1.79,3.32,0.00,0.00,1.79,0.00,...,0.00,4.86,0.00,0.00,2.81,2.84,0.00,0.00,1.08,0.00
19645,C3orf79,152118,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.07,0.00,0.00
19646,CTD-2116N17.1,0,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


In [107]:
############################# 1.Test in case samples are paired (related) #####################################
paired=[]
for i in range (len(healthy)):
    pv=ttest_rel(healthy.iloc[i,2:],cancerous.iloc[i,2:]).pvalue
    paired.append(pv)
paired
len(paired)

17643

In [109]:
# FDR Correction 
# multipletests on the list result Nan so I converted it to an array and used the mask property to get finit numbers

corrected_paired=multipletests(paired, alpha=0.05, method='fdr_bh')[1]
corrected_paired

array([5.55401729e-08, 3.00349169e-01, 2.77185268e-04, ...,
       2.90682352e-01, 7.58268996e-06, 1.10465640e-22])

In [110]:
# DEGs before and after the FDR correction in a data frame
significance_genes = pd.DataFrame({'Gene_name':healthy.iloc[:,0], 'p-values':paired, 'p-values_fdr':corrected_paired})
significance_genes

,Gene_name,p-values,p-values_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08
1,LIN7B,2.891646e-01,3.003492e-01
2,LXN,2.322367e-04,2.771853e-04
3,CNKSR2,3.420577e-12,5.489789e-12
4,SCML1,6.251346e-02,6.751500e-02
...,...,...,...
19643,HAVCR2,8.644836e-21,2.155772e-20
19644,RP1-66C13.4,4.081896e-03,4.642358e-03
19645,C3orf79,2.796271e-01,2.906824e-01
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06


In [111]:
#Compare the two DEGs sets 
significance_genes['significance:p_vlaue'] = significance_genes['p-values'].apply(lambda x: x < 0.05)
significance_genes['significance:p_vlaue_fdr'] = significance_genes['p-values_fdr'].apply(lambda x: x < 0.05)
significance_genes

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08,True,True
1,LIN7B,2.891646e-01,3.003492e-01,False,False
2,LXN,2.322367e-04,2.771853e-04,True,True
3,CNKSR2,3.420577e-12,5.489789e-12,True,True
4,SCML1,6.251346e-02,6.751500e-02,False,False
...,...,...,...,...,...
19643,HAVCR2,8.644836e-21,2.155772e-20,True,True
19644,RP1-66C13.4,4.081896e-03,4.642358e-03,True,True
19645,C3orf79,2.796271e-01,2.906824e-01,False,False
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06,True,True


In [112]:
#the common and distinct genes
diffrentially_genes = significance_genes[significance_genes['significance:p_vlaue_fdr']== True]
diffrentially_genes

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08,True,True
2,LXN,2.322367e-04,2.771853e-04,True,True
3,CNKSR2,3.420577e-12,5.489789e-12,True,True
6,GSDMD,3.041721e-06,3.890466e-06,True,True
7,AKR1C1,1.938575e-05,2.406408e-05,True,True
...,...,...,...,...,...
19642,SPINT2,3.034629e-34,3.244846e-33,True,True
19643,HAVCR2,8.644836e-21,2.155772e-20,True,True
19644,RP1-66C13.4,4.081896e-03,4.642358e-03,True,True
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06,True,True


In [113]:
Gens=diffrentially_genes['Gene_name'].to_list()
len(Gens)
#print(Gens)

16245

In [117]:
#################################### 2.Test in case samples are independent #############################################
indep=[]
for i in range (len(healthy)):
    pv=ttest_rel(healthy.iloc[i,2:],cancerous.iloc[i,2:]).pvalue
    indep.append(pv)
#indep
len(healthy)

17643

In [118]:
# FDR Correction 
# multipletests on the list result Nan so I converted it to an array and used the mask property to get finit numbers
indep_array=np.array(indep)
corrected_indep=multipletests(indep, alpha=0.05, method='fdr_bh')[1]
corrected_indep

array([5.55401729e-08, 3.00349169e-01, 2.77185268e-04, ...,
       2.90682352e-01, 7.58268996e-06, 1.10465640e-22])

In [119]:
# DEGs before and after the FDR correction in a data frame
significance_indep= pd.DataFrame({'Gene_name':healthy.iloc[:,0], 'p-values':indep, 'p-values_fdr':corrected_indep})
significance_indep

,Gene_name,p-values,p-values_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08
1,LIN7B,2.891646e-01,3.003492e-01
2,LXN,2.322367e-04,2.771853e-04
3,CNKSR2,3.420577e-12,5.489789e-12
4,SCML1,6.251346e-02,6.751500e-02
...,...,...,...
19643,HAVCR2,8.644836e-21,2.155772e-20
19644,RP1-66C13.4,4.081896e-03,4.642358e-03
19645,C3orf79,2.796271e-01,2.906824e-01
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06


In [120]:
#Compare the two DEGs sets 
significance_indep['significance:p_vlaue'] = significance_indep['p-values'].apply(lambda x: x < 0.05)
significance_indep['significance:p_vlaue_fdr'] = significance_indep['p-values_fdr'].apply(lambda x: x < 0.05)
significance_indep

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08,True,True
1,LIN7B,2.891646e-01,3.003492e-01,False,False
2,LXN,2.322367e-04,2.771853e-04,True,True
3,CNKSR2,3.420577e-12,5.489789e-12,True,True
4,SCML1,6.251346e-02,6.751500e-02,False,False
...,...,...,...,...,...
19643,HAVCR2,8.644836e-21,2.155772e-20,True,True
19644,RP1-66C13.4,4.081896e-03,4.642358e-03,True,True
19645,C3orf79,2.796271e-01,2.906824e-01,False,False
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06,True,True


In [121]:
#the common and distinct genes
diffrentially_indep = significance_indep[significance_indep['significance:p_vlaue_fdr']== True]
diffrentially_indep

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,5.554017e-08,True,True
2,LXN,2.322367e-04,2.771853e-04,True,True
3,CNKSR2,3.420577e-12,5.489789e-12,True,True
6,GSDMD,3.041721e-06,3.890466e-06,True,True
7,AKR1C1,1.938575e-05,2.406408e-05,True,True
...,...,...,...,...,...
19642,SPINT2,3.034629e-34,3.244846e-33,True,True
19643,HAVCR2,8.644836e-21,2.155772e-20,True,True
19644,RP1-66C13.4,4.081896e-03,4.642358e-03,True,True
19646,CTD-2116N17.1,5.990768e-06,7.582690e-06,True,True


In [123]:
Gens=diffrentially_indep['Gene_name'].to_list()
len(Gens)
#print(Gens)

16245